In [6]:
import os
import yaml
import logging
import platform
import argparse
from tqdm import tqdm
import os
import json
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

import torch
from torch import optim
import torch.utils.data
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from utils.model import VAE
from utils.dataset import OutlierDataset

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [7]:
def load_csv(config) -> pd.DataFrame:
        """
        - csv파일 데이터프레임으로 불러오기.
        - value를 제외한 모든 칼럼을 무시.
        """
        data_path = config['data_path']
        try:
            data = pd.read_csv(data_path, usecols=['value'])
        except Exception as e:
            print(f"csv 파일을 불러오는 도중 문제가 발생했습니다: {e}")
            print("config.yaml을 열어 csv 파일 경로가 상대경로로 기입되어 있는지 확인하세요.")
        return data
    
def load_json(config) -> list[list[int]]:
    """
    - 비교적 이상치의 위험성이 적은 구간에 대한 정보를 담은 json 파일 불러오기.
    - csv 파일 경로를 파싱해서 json의 키 값으로 재활용한다.
    """
    data_path = config['data_path']
    interval_path = config['interval_path']
    dirc_name = data_path.split('/')[1]
    file_name = data_path.split('/')[2]
    try:
        with open(interval_path, 'r') as file:
            interval = json.load(file)[dirc_name][file_name]
    except Exception as e:
        print(f"json 파일을 불러오는 도중 문제가 발생했습니다: {e}")
        print("config.yaml을 열어 csv, json 파일 경로가 상대경로로 기입되어 있는지 확인하세요.")
    return interval

In [9]:
data = load_csv(config)
intervals = load_json(config)

In [10]:
data

,value
0,31.9516
1,32.7770
2,32.6126
3,32.4761
4,32.6497
...,...
30116,32.4947
30117,32.1726
30118,32.0665
30119,32.0694


In [11]:
intervals

[[0, 1000], [2000, 3000]]

In [12]:
def split_by_interval(data: pd.DataFrame,
                          intervals: list[list[int]]) -> list[pd.DataFrame]:
        """
        - 불러온 구간 정보로 원본 데이터프레임 분할, 분할한 데이터프레임을 리스트에 담아 반환.
        - 각각의 분할 구간은 우리가 훈련 데이터로 사용할 수 있는 값들을 의미한다.
        """
        subsets = []
        for start, end in intervals:
            subset = data.iloc[start:end]
            subsets.append(subset)
        return subsets

def slice_by_window(config, data: list[pd.DataFrame]) -> list[np.ndarray]:
    """
    - 분할된 데이터프레임별로 주어진 윈도우로 스텝 사이즈 만큼 이동하며 데이터 추출.
    - 만약에 주어진 구간 안에서 윈도우 설정이 불가능하면 해당 구간을 건너뛴다.
    """
    window_size = config['seq_size']
    step_size = config['step_size']
    windows = []
    for subset in data:
        values = subset['value'].to_numpy()
        for start in range(0, len(values) - window_size + 1, step_size):
            window = values[start:start + window_size]
            windows.append(window)
    return windows

In [13]:
data = split_by_interval(data, intervals)
data

[       value
 0    31.9516
 1    32.7770
 2    32.6126
 3    32.4761
 4    32.6497
 ..       ...
 995  31.5987
 996  31.9012
 997  31.6645
 998  31.8105
 999  31.8217
 
 [1000 rows x 1 columns],
         value
 2000  31.7199
 2001  32.2450
 2002  31.6883
 2003  31.8868
 2004  32.0465
 ...       ...
 2995  32.8189
 2996  33.0118
 2997  32.7250
 2998  33.0652
 2999  32.6277
 
 [1000 rows x 1 columns]]

In [15]:
data = slice_by_window(config, data)

In [18]:
len(data)

36

In [19]:
data[0].shape

(150,)

In [20]:
def standardize(data: list[np.ndarray]) -> torch.Tensor:
        """
        - 모든 값들을 시퀀스 단위로 정규화, [seq_size, batch_size, 1] 차원의 텐서로 반환.
        """
        data_array = np.array(data)
        means = data_array.mean(axis=1, keepdims=True)
        stds = data_array.std(axis=1, keepdims=True)
        normalized_data = (data_array - means) / stds
        return torch.tensor(normalized_data, dtype=torch.float32).unsqueeze(-1)

In [21]:
data = standardize(data)
data

tensor([[[-0.6897],
         [ 1.4473],
         [ 1.0216],
         ...,
         [ 0.8738],
         [ 1.5646],
         [-0.7659]],

        [[ 0.6821],
         [ 1.5753],
         [ 0.1827],
         ...,
         [ 0.2915],
         [-1.4896],
         [-1.1182]],

        [[ 0.5173],
         [ 0.0119],
         [ 0.2152],
         ...,
         [-0.7234],
         [ 0.5486],
         [-0.3418]],

        ...,

        [[-0.6360],
         [-1.1288],
         [-2.2980],
         ...,
         [ 0.2688],
         [ 1.2236],
         [-1.4799]],

        [[ 0.0577],
         [-1.1270],
         [-0.7965],
         ...,
         [ 0.4339],
         [ 0.4881],
         [ 2.7353]],

        [[-0.1102],
         [-0.6594],
         [-0.0785],
         ...,
         [ 0.1016],
         [ 1.1396],
         [-0.1953]]])

In [23]:
data.shape

torch.Size([36, 150, 1])